In [2]:

import sys
import os
import sys
file = "/Users/charlesmiller/Documents/Code/dynamics_lab/DSDL/DLAESINDy/testcases/parallel_sweep.ipynb"
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(file))))

# sys.path.append("../aesindy")
from default_params import default_params
import numpy as np
from aesindy.solvers import DatasetConstructor
from aesindy.training import TrainModel
from aesindy.helper_functions import call_polygon
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint
import tensorflow as tf
import gc


def update_params_from_wandb(params, wandb_config):
    """
    Update the default parameters with values from the wandb config.
    
    Args:
    default_params (dict): The default parameter dictionary
    wandb_config (wandb.config): The wandb config object
    
    Returns:
    dict: Updated parameter dictionary
    """
    updated_params = params.copy()
    
    for key, value in wandb_config.items():
        if key in updated_params:
            updated_params[key] = value
        else:
            print(f"Warning: '{key}' found in wandb config but not in default params. Ignoring.")
    
    return updated_params



def model_runner(wandb_params, raw_data):
    params = update_params_from_wandb(default_params, wandb_params)
    params['model'] = 'qqq'
    params['case'] = 'hyp'
    params['use_wandb'] = True
    params['prediciton_mode'] = 'max'
    print(params)
    ## slice the data based on a fractional proportion, must remain in sequential order
    raw_data = raw_data[int(params['data_length']*len(raw_data)):]
    data_dict = {
        'x':[raw_data['x'].values],
        'dt': 900
        }
    data_builder = DatasetConstructor(input_dim=params['input_dim'],
                    interpolate=params['interpolate'],
                    interp_dt=params['interp_dt'],
                    savgol_interp_coefs=params['interp_coefs'],
                    interp_kind=params['interp_kind'],
                    future_steps=params['future_steps'])
    data_builder.build_solution(data_dict)
    train_data = data_builder.get_data()
    trainer = TrainModel(train_data, params)
    trainer.fit() 

    del trainer.model
    del trainer

    tf.keras.backend.clear_session()
    gc.collect()

def wandb_sweep(data):


    # Hyperparameters
    sweep_config = {
        'method': 'bayes', 
        'program': '/Users/charlesmiller/Documents/Code/IIE/dynamics_lab/DSDL/DLAESINDy/testcases/hyperparam_run.pycd',
        'bayes': {
            'bayes_initial_samples': 75,
            'exploration_factor': 0.2,
        },
        'metric': {
            'name': 'current_best_val_prediction_loss',
            'goal': 'minimize'
        },
        "parameters": {
            "learning_rate": {'values': [0.001,.0001]},
            "latent_dim": {'values': [2,3,4,6]},
            "input_dim": {'values': [64,128,256]},
            "poly_order": {'values': [2,3]},
            "n_frequencies": {'values': [2,3,4]},
            "loss_weight_layer_l2": {'values': [.0,0.05]},
            "loss_weight_x0": {'values': [0.03,0.05]},
            "loss_weight_integral": {'values': [0.01,0.05,0.1]},
            "loss_weight_sindy_regularization": {'values': [1e-5,1e-3,1e-1]},
            "loss_weight_prediction": {'values': [0.5,1.0]},
            "loss_weight_rec": {'values': [0.3,0.6,0.9]},
            "loss_weight_sindy_z": {'values': [0.001,0.0001]},
            "loss_weight_sindy_x": {'values': [0.0003,0.0001]},
            "batch_size": {'values': [32,64]},
            "data_length": {'values': [0.5,.75]},
            "widths_ratios": {'values': [[0.5,0.25],[0.75,0.5,0.25],[0.8,0.6,0.4,0.2]]},
            "sindy_threshold": {'values': [0.2,0.3]},
            "sindy_init_scale": {'values': [3.0,5.0]},
            "threshold_frequency": {'values': [55,80]},
            "coefficient_threshold": {'values': [1,2,3,4]},
            "sindycall_freq": {'values': [55,80]},
        }
    }

    # Initialize the sweep
    sweep_id = 'cfm_inv/DLAESINDy/7ni74345'

    # Define the objective function for the sweep
    def sweep_train():

        wandb.init(project="DLAESINDy", config=wandb.config)
        model_runner(wandb.config, data)

    # Start the sweep
    wandb.agent(sweep_id, function=sweep_train, count=1000) 


In [3]:
raw_data = call_polygon('SPY','2012-01-01','2024-06-01','minute',15)
raw_data = raw_data[['c']]
raw_data = raw_data.rename(columns={'c':'x'})
wandb_sweep(raw_data)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
404 response executing GraphQL.
{"errors":[{"message":"could not find project cfm_inv/uncategorized during createAgent","path":["createAgent"]}],"data":{"createAgent":null}}
wandb: ERROR Error while calling W&B API: could not find project cfm_inv/uncategorized during createAgent (<Response [404]>)


UsageError: could not find project cfm_inv/uncategorized during createAgent